In [ ]:
N = 512 * 8

np.random.seed(4)

# set the random seed and create the sobol sequence generator
sobol_engine = scipy.stats.qmc.Sobol( d=4, seed=4 )

# get M initial states spaced in the recommended ranges
#X = X_sobol = initial_states = (sobol_engine.random_base2( m=int(np.log2(N)) ) - 0.5 ) * 2 * np.array( [ 10, 20, np.pi, 20 ] )
X = X_sobol = initial_states = (sobol_engine.random_base2( m=int(np.log2(N)) ) - 0.5 ) * 2 * np.array( [ 5, 2, 1, 2 ] ) + np.array([0,0,np.pi,0])
#X[:,2] = np.array( [ CartPole.remap_angle(theta) for theta in X[:,2] ] )

# generate the changes in state from these states
Y = np.array( [ CartPole.perform_action( state ) - state for state in initial_states ] )


M = 32 * 32
kernel_centres = X[:M]

# loop over the kernel centres and evaluate the K function across all the Xs at each
Kmn = np.zeros( (M,N) )
for i, kernel_centre in enumerate( kernel_centres ):

    Kmn[i] = K( X, kernel_centre[np.newaxis] )

# same as above but only use first M vectors from X
Kmm = np.zeros( (M,M) )
for i, kernel_centre in enumerate( kernel_centres ):

    Kmm[i] = K( X[:M], kernel_centre[np.newaxis] )


l = 1e-4

KmnKnm = Kmn @ Kmn.T
a = KmnKnm + l * Kmm
b = Kmn @ Y

alpha_m = np.linalg.lstsq( a, b, rcond=None )[0]

def nonlinear_model( state ):
    
    kernels = K( state[np.newaxis], kernel_centres )
    
    weighted_sums = kernels @ alpha_m
    
    return weighted_sums.squeeze()

#kernel_centres = cache["kernel_centres_fitted"]
#alpha_m = cache["alpha_m_fitted"]

fig, ax = plt.subplots(1, 1, num=14, figsize=(10,2.5))
fig.subplots_adjust(bottom=0.2, right=0.99, left=0.1)
sf3utility.setup_phase_portrait( ax )

# small oscillations about stable equilibrium

predicted_state = np.array( [0, 0.126, np.pi, 1] )
actual_state = np.array( [0, 0.126, np.pi, 1] )

prediction_states = []
actual_states = []
time = []

for i in range(100):
    
    prediction_states.append( predicted_state )
    actual_states.append( actual_state )
    time.append( i * 0.2 )
    
    predicted_state = nonlinear_model( predicted_state ) + predicted_state
    actual_state    = CartPole.perform_action( actual_state )

prediction_states = np.array( prediction_states )
actual_states = np.array( actual_states )
time = np.array( time )

x = time
y1 = prediction_states[:,1]
y2 = actual_states[:,1]


f, u = scipy.interpolate.splprep( [x, y1], s=0, per=True )
xint, y1int = scipy.interpolate.splev(np.linspace(0, 1, 10000), f)
xint, y1int = xint[:5000], y1int[:5000]

f, u = scipy.interpolate.splprep( [x, y2], s=0, per=True )
_, y2int = scipy.interpolate.splev(np.linspace(0, 1, 10000), f)
y2int = y2int[:5000]

ax.plot( xint, y2int, color="orange" )
ax.plot( xint, y1int, color="magenta" )

ax.set_title("Specialised Model Time Evolution Prediction vs. Actual")
ax.set_xlabel("Seconds Elapsed")
ax.set_ylabel("Cart Velocity")